In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [2]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,beni bugun de unutmayip mesaj attigin icin tes...,1
1,i̇yi ki kontr aldım... dün geceden beri aynı a...,0
2,turkcell 1tl min ustune yatip 50mb verdi hepsi...,0
3,canini yedigimin turkcell i sende olmasan bizi...,1
4,halkın takımı oldugunu iddia eden 8taş turkcel...,0
...,...,...
9121,turkcell kalitesi diye bişey var,1
9122,bugun de turkcell ile skinti yasayarak basladi...,0
9123,zaten turkcell'e çalışıyoruz aq ?????????? @ g...,0
9124,vodafone'a geçmeyeyim diye turkcell üzerime he...,1


In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [6]:
# Load my word embeddings
#import gensim
#wordembeddings = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/CNNPhraseEmbeddings/Resources/GoogleNews-vectors-negative300.bin', binary=True)

import fasttext
import fasttext.util
wordembeddings = fasttext.load_model('D:/Desktop/İndirilenler yedek/cc.tr.300.bin')

In [7]:
def fasttext (X_train,y_train,X_test,y_test):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    sequences_X_train = tokenizer.texts_to_sequences(X_train)
    word_index = tokenizer.word_index
    max_length = 0
    for review_number in range(len(sequences_X_train)):
        numberofwords=len(sequences_X_train[review_number])
        if (numberofwords) > (max_length):
            max_length = numberofwords

    X_train = pad_sequences(sequences_X_train, maxlen=max_length)
    y_train = np.asarray(y_train)

    sequences_X_test = tokenizer.texts_to_sequences(X_test)
    X_test = pad_sequences(sequences_X_test, maxlen=max_length)
    y_test = np.asarray(y_test)

    unique_words = len(word_index)
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 300
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items():
        try:
            embedding_vector = wordembeddings[word]
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
      
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [8]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
    
    pat = 5
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(128,activation='relu',return_sequences= True))
    model.add(SimpleRNN(256,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 10s 37ms/step - loss: 0.5729 - acc: 0.6966 - precision: 0.7087 - recall: 0.6662
Epoch 2/50
257/257 [==============================] - 9s 36ms/step - loss: 0.4473 - acc: 0.8017 - precision: 0.8232 - recall: 0.7676
Epoch 3/50
257/257 [==============================] - 9s 36ms/step - loss: 0.3945 - acc: 0.8258 - precision: 0.8387 - recall: 0.8061
Epoch 4/50
257/257 [==============================] - 10s 37ms/step - loss: 0.3418 - acc: 0.8555 - precision: 0.8657 - recall: 0.8410
Epoch 5/50
257/257 [==============================] - 9s 37ms/step - loss: 0.3060 - acc: 0.8773 - precision: 0.8898 - recall: 0.8608
Epoch 6/50
257/257 [==============================] - 9s 36ms/step - loss: 0.2521 - acc: 0.8977 - precision: 0.9075 - recall: 0.8854
Epoch 7/50
257/257 [==============================] - 9s 34ms/step - loss: 0.2047 - acc: 0.9176 - preci

257/257 [==============================] - 9s 36ms/step - loss: 0.0418 - acc: 0.9871 - precision_1: 0.9887 - recall_1: 0.9854
Epoch 26/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0401 - acc: 0.9873 - precision_1: 0.9878 - recall_1: 0.9868
Epoch 27/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0283 - acc: 0.9918 - precision_1: 0.9919 - recall_1: 0.9917
Epoch 28/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0492 - acc: 0.9845 - precision_1: 0.9858 - recall_1: 0.9832
Epoch 00028: early stopping
29/29 [==============================] - 0s 16ms/step - loss: 0.9810 - acc: 0.7952 - precision_1: 0.7869 - recall_1: 0.8223
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/50
257/257 [==============================] - 9s 36ms/step - loss: 0.5492 - acc: 0.7285 - precision_2: 0.7467 - recall_2: 0.6929
Epoch 2/50
257/257 [==============================] - 9s 36ms/step - loss:

257/257 [==============================] - 9s 36ms/step - loss: 0.0318 - acc: 0.9909 - precision_3: 0.9900 - recall_3: 0.9917
Epoch 24/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0121 - acc: 0.9966 - precision_3: 0.9963 - recall_3: 0.9968
Epoch 25/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0238 - acc: 0.9918 - precision_3: 0.9936 - recall_3: 0.9900
Epoch 26/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0300 - acc: 0.9904 - precision_3: 0.9895 - recall_3: 0.9912
Epoch 27/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0554 - acc: 0.9833 - precision_3: 0.9838 - recall_3: 0.9826
Epoch 28/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0278 - acc: 0.9910 - precision_3: 0.9912 - recall_3: 0.9907
Epoch 29/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0369 - acc: 0.9875 - precision_3: 0.9870 - recall_3: 0.9878
Epoch 00029: early stopping
29/29 [===========

257/257 [==============================] - 10s 37ms/step - loss: 0.0451 - acc: 0.9858 - precision_5: 0.9861 - recall_5: 0.9854
Epoch 00019: early stopping
29/29 [==============================] - 0s 16ms/step - loss: 0.9311 - acc: 0.7809 - precision_5: 0.7533 - recall_5: 0.8501
------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/50
257/257 [==============================] - 10s 37ms/step - loss: 0.5644 - acc: 0.7180 - precision_6: 0.7304 - recall_6: 0.6920
Epoch 2/50
257/257 [==============================] - 9s 36ms/step - loss: 0.4418 - acc: 0.8016 - precision_6: 0.8164 - recall_6: 0.7786
Epoch 3/50
257/257 [==============================] - 9s 37ms/step - loss: 0.3914 - acc: 0.8318 - precision_6: 0.8428 - recall_6: 0.8161
Epoch 4/50
257/257 [==============================] - 9s 37ms/step - loss: 0.3432 - acc: 0.8557 - precision_6: 0.8602 - recall_6: 0.8499
Epoch 5/50
257/257 [==============================] - 9s 36ms/step - loss: 

257/257 [==============================] - 10s 38ms/step - loss: 0.0555 - acc: 0.9830 - precision_7: 0.9826 - recall_7: 0.9835
Epoch 26/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0267 - acc: 0.9915 - precision_7: 0.9913 - recall_7: 0.9918
Epoch 27/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0315 - acc: 0.9892 - precision_7: 0.9893 - recall_7: 0.9891
Epoch 28/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0285 - acc: 0.9906 - precision_7: 0.9915 - recall_7: 0.9898
Epoch 00028: early stopping
29/29 [==============================] - 0s 16ms/step - loss: 0.9521 - acc: 0.7982 - precision_7: 0.7540 - recall_7: 0.8636
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/50
257/257 [==============================] - 9s 36ms/step - loss: 0.5505 - acc: 0.7204 - precision_8: 0.7253 - recall_8: 0.7100
Epoch 2/50
257/257 [==============================] - 9s 35ms/step - loss

257/257 [==============================] - 10s 40ms/step - loss: 0.0268 - acc: 0.9915 - precision_9: 0.9924 - recall_9: 0.9905
Epoch 24/50
257/257 [==============================] - 10s 37ms/step - loss: 0.0330 - acc: 0.9905 - precision_9: 0.9914 - recall_9: 0.9895
Epoch 25/50
257/257 [==============================] - 10s 37ms/step - loss: 0.0207 - acc: 0.9925 - precision_9: 0.9924 - recall_9: 0.9924
Epoch 26/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0577 - acc: 0.9805 - precision_9: 0.9805 - recall_9: 0.9805
Epoch 27/50
257/257 [==============================] - 10s 38ms/step - loss: 0.0112 - acc: 0.9968 - precision_9: 0.9968 - recall_9: 0.9968
Epoch 28/50
257/257 [==============================] - 10s 38ms/step - loss: 0.0213 - acc: 0.9932 - precision_9: 0.9929 - recall_9: 0.9934
Epoch 29/50
257/257 [==============================] - 10s 37ms/step - loss: 0.0482 - acc: 0.9860 - precision_9: 0.9849 - recall_9: 0.9871
Epoch 30/50
257/257 [===================

In [9]:
def Average(lst):
    return sum(lst) / len(lst)

In [10]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.784457
test precision: 0.763775
test recall: 0.826143
test f1_score: 0.793087


In [11]:
#cnn

In [12]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn

    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=256, kernel_size=5, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=2)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(360, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(300, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(260, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(150, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(120, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(80, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(1, activation='sigmoid'))
    modelCNN.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNN.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 12s 48ms/step - loss: 0.6352 - acc: 0.6302 - precision_10: 0.6370 - recall_10: 0.5938
Epoch 2/50
257/257 [==============================] - 12s 48ms/step - loss: 0.5179 - acc: 0.7545 - precision_10: 0.7799 - recall_10: 0.7046
Epoch 3/50
257/257 [==============================] - 13s 49ms/step - loss: 0.4439 - acc: 0.7991 - precision_10: 0.8165 - recall_10: 0.7681
Epoch 4/50
257/257 [==============================] - 12s 48ms/step - loss: 0.3862 - acc: 0.8287 - precision_10: 0.8449 - recall_10: 0.8024
Epoch 5/50
257/257 [==============================] - 12s 48ms/step - loss: 0.3335 - acc: 0.8592 - precision_10: 0.8680 - recall_10: 0.8451
Epoch 6/50
257/257 [==============================] - 12s 48ms/step - loss: 0.2810 - acc: 0.8802 - precision_10: 0.8870 - recall_10: 0.8696
Epoch 7/50
257/257 [==============================] - 12s 48ms/

257/257 [==============================] - 12s 48ms/step - loss: 0.0562 - acc: 0.9791 - precision_11: 0.9805 - recall_11: 0.9781
Epoch 26/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0563 - acc: 0.9802 - precision_11: 0.9789 - recall_11: 0.9820
Epoch 27/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0484 - acc: 0.9817 - precision_11: 0.9820 - recall_11: 0.9820
Epoch 28/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0539 - acc: 0.9814 - precision_11: 0.9824 - recall_11: 0.9808
Epoch 29/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0550 - acc: 0.9808 - precision_11: 0.9803 - recall_11: 0.9817
Epoch 30/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0477 - acc: 0.9830 - precision_11: 0.9839 - recall_11: 0.9825
Epoch 31/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0517 - acc: 0.9810 - precision_11: 0.9820 - recall_11: 0.9805
Epoch 32/50
257/257 [====

257/257 [==============================] - 12s 46ms/step - loss: 0.0557 - acc: 0.9810 - precision_12: 0.9794 - recall_12: 0.9827
Epoch 27/50
257/257 [==============================] - 12s 46ms/step - loss: 0.0577 - acc: 0.9803 - precision_12: 0.9815 - recall_12: 0.9791 - ETA: 0s - loss: 0.0571 - acc: 0.9802 - precision_12: 0.98
Epoch 28/50
257/257 [==============================] - 12s 46ms/step - loss: 0.0578 - acc: 0.9786 - precision_12: 0.9807 - recall_12: 0.9764
Epoch 29/50
257/257 [==============================] - 12s 45ms/step - loss: 0.0562 - acc: 0.9786 - precision_12: 0.9776 - recall_12: 0.9796
Epoch 30/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0449 - acc: 0.9844 - precision_12: 0.9842 - recall_12: 0.9847
Epoch 31/50
257/257 [==============================] - 12s 46ms/step - loss: 0.0471 - acc: 0.9811 - precision_12: 0.9803 - recall_12: 0.9820
Epoch 32/50
257/257 [==============================] - 13s 49ms/step - loss: 0.0495 - acc: 0.9832 - precis

257/257 [==============================] - 12s 45ms/step - loss: 0.0458 - acc: 0.9840 - precision_13: 0.9827 - recall_13: 0.9853
Epoch 36/50
257/257 [==============================] - 12s 46ms/step - loss: 0.0386 - acc: 0.9864 - precision_13: 0.9847 - recall_13: 0.9880
Epoch 37/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0448 - acc: 0.9861 - precision_13: 0.9873 - recall_13: 0.9849
Epoch 38/50
257/257 [==============================] - 12s 46ms/step - loss: 0.0393 - acc: 0.9858 - precision_13: 0.9844 - recall_13: 0.9871
Epoch 39/50
257/257 [==============================] - 12s 45ms/step - loss: 0.0504 - acc: 0.9826 - precision_13: 0.9826 - recall_13: 0.98242s - loss:
Epoch 40/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0413 - acc: 0.9847 - precision_13: 0.9858 - recall_13: 0.9834
Epoch 41/50
257/257 [==============================] - 12s 46ms/step - loss: 0.0525 - acc: 0.9833 - precision_13: 0.9822 - recall_13: 0.9844
Epoch 00041: ea

257/257 [==============================] - 12s 47ms/step - loss: 0.2913 - acc: 0.8835 - precision_15: 0.8914 - recall_15: 0.8725
Epoch 6/50
257/257 [==============================] - 12s 46ms/step - loss: 0.2449 - acc: 0.9022 - precision_15: 0.9096 - recall_15: 0.89251s - loss: 0.2452 - acc: 0.9031 
Epoch 7/50
257/257 [==============================] - 12s 48ms/step - loss: 0.2001 - acc: 0.9239 - precision_15: 0.9261 - recall_15: 0.9209
Epoch 8/50
257/257 [==============================] - 12s 46ms/step - loss: 0.1609 - acc: 0.9369 - precision_15: 0.9355 - recall_15: 0.93828s - loss: 0 - ETA: 5s - loss: 0.1505 - acc: 0.9413 
Epoch 9/50
257/257 [==============================] - 12s 47ms/step - loss: 0.1410 - acc: 0.9472 - precision_15: 0.9483 - recall_15: 0.9455
Epoch 10/50
257/257 [==============================] - 12s 47ms/step - loss: 0.1316 - acc: 0.9486 - precision_15: 0.9507 - recall_15: 0.9460
Epoch 11/50
257/257 [==============================] - 12s 47ms/step - loss: 0.1170 - 

257/257 [==============================] - 12s 47ms/step - loss: 0.0565 - acc: 0.9810 - precision_16: 0.9817 - recall_16: 0.9803
Epoch 30/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0422 - acc: 0.9848 - precision_16: 0.9854 - recall_16: 0.9842
Epoch 31/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0526 - acc: 0.9800 - precision_16: 0.9796 - recall_16: 0.9805
Epoch 32/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0556 - acc: 0.9787 - precision_16: 0.9788 - recall_16: 0.9786
Epoch 33/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0472 - acc: 0.9821 - precision_16: 0.9834 - recall_16: 0.9808
Epoch 34/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0475 - acc: 0.9826 - precision_16: 0.9822 - recall_16: 0.9829
Epoch 35/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0443 - acc: 0.9855 - precision_16: 0.9854 - recall_16: 0.9856
Epoch 00035: early stoppi

257/257 [==============================] - 12s 48ms/step - loss: 0.0925 - acc: 0.9651 - precision_18: 0.9655 - recall_18: 0.9643
Epoch 15/50
257/257 [==============================] - 13s 49ms/step - loss: 0.0874 - acc: 0.9693 - precision_18: 0.9690 - recall_18: 0.9694
Epoch 16/50
257/257 [==============================] - 13s 51ms/step - loss: 0.0972 - acc: 0.9653 - precision_18: 0.9646 - recall_18: 0.9658
Epoch 17/50
257/257 [==============================] - 13s 51ms/step - loss: 0.0846 - acc: 0.9685 - precision_18: 0.9708 - recall_18: 0.9658
Epoch 18/50
257/257 [==============================] - 13s 50ms/step - loss: 0.0792 - acc: 0.9710 - precision_18: 0.9716 - recall_18: 0.9702
Epoch 19/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0786 - acc: 0.9705 - precision_18: 0.9707 - recall_18: 0.97021s - loss: 0.0735 - acc: 0.9715 - prec
Epoch 20/50
257/257 [==============================] - 13s 51ms/step - loss: 0.0643 - acc: 0.9770 - precision_18: 0.9770 - recal

257/257 [==============================] - 12s 48ms/step - loss: 0.0899 - acc: 0.9677 - precision_19: 0.9684 - recall_19: 0.9667
Epoch 18/50
257/257 [==============================] - 13s 50ms/step - loss: 0.0849 - acc: 0.9690 - precision_19: 0.9673 - recall_19: 0.97041s - loss: 0.0863 
Epoch 19/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0729 - acc: 0.9724 - precision_19: 0.9712 - recall_19: 0.9733
Epoch 20/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0734 - acc: 0.9741 - precision_19: 0.9731 - recall_19: 0.9748
Epoch 21/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0674 - acc: 0.9749 - precision_19: 0.9748 - recall_19: 0.9748
Epoch 22/50
257/257 [==============================] - 12s 48ms/step - loss: 0.0681 - acc: 0.9739 - precision_19: 0.9755 - recall_19: 0.9721
Epoch 23/50
257/257 [==============================] - 12s 47ms/step - loss: 0.0589 - acc: 0.9778 - precision_19: 0.9763 - recall_19: 0.9792
Epoch 2

In [13]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.760356
test precision: 0.765662
test recall: 0.752952
test f1_score: 0.757889


In [14]:
#lstm

In [15]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
    

    modelLSTM = Sequential()
    modelLSTM.add(embedding_layer)
    modelLSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    modelLSTM.add(Dense(1, activation='sigmoid'))
    modelLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 25s 97ms/step - loss: 0.5581 - acc: 0.7176 - precision_20: 0.7373 - recall_20: 0.6791
Epoch 2/50
257/257 [==============================] - 25s 97ms/step - loss: 0.4805 - acc: 0.7760 - precision_20: 0.7907 - recall_20: 0.7527
Epoch 3/50
257/257 [==============================] - 25s 98ms/step - loss: 0.4570 - acc: 0.7907 - precision_20: 0.8048 - recall_20: 0.7694
Epoch 4/50
257/257 [==============================] - 25s 98ms/step - loss: 0.4401 - acc: 0.7989 - precision_20: 0.8104 - recall_20: 0.7820
Epoch 5/50
257/257 [==============================] - 25s 97ms/step - loss: 0.4168 - acc: 0.8088 - precision_20: 0.8199 - recall_20: 0.7932
Epoch 6/50
257/257 [==============================] - 25s 97ms/step - loss: 0.4023 - acc: 0.8155 - precision_20: 0.8249 - recall_20: 0.8027
Epoch 7/50
257/257 [==============================] - 25s 98ms/

257/257 [==============================] - 25s 98ms/step - loss: 0.4062 - acc: 0.8081 - precision_21: 0.8130 - recall_21: 0.8011
Epoch 7/50
257/257 [==============================] - 25s 98ms/step - loss: 0.3941 - acc: 0.8182 - precision_21: 0.8332 - recall_21: 0.7965
Epoch 8/50
257/257 [==============================] - 25s 98ms/step - loss: 0.3761 - acc: 0.8297 - precision_21: 0.8366 - recall_21: 0.8201
Epoch 9/50
257/257 [==============================] - 25s 97ms/step - loss: 0.3631 - acc: 0.8365 - precision_21: 0.8437 - recall_21: 0.82662s - loss: 0.3
Epoch 10/50
257/257 [==============================] - 25s 98ms/step - loss: 0.3484 - acc: 0.8385 - precision_21: 0.8436 - recall_21: 0.83182s - loss: 0
Epoch 11/50
257/257 [==============================] - 26s 100ms/step - loss: 0.3391 - acc: 0.8491 - precision_21: 0.8545 - recall_21: 0.8422
Epoch 12/50
257/257 [==============================] - 26s 102ms/step - loss: 0.3171 - acc: 0.8573 - precision_21: 0.8652 - recall_21: 0.8471


257/257 [==============================] - 26s 101ms/step - loss: 0.3239 - acc: 0.8546 - precision_22: 0.8591 - recall_22: 0.8461
Epoch 12/50
257/257 [==============================] - 25s 96ms/step - loss: 0.3130 - acc: 0.8608 - precision_22: 0.8654 - recall_22: 0.8525
Epoch 13/50
257/257 [==============================] - 25s 97ms/step - loss: 0.3017 - acc: 0.8684 - precision_22: 0.8746 - recall_22: 0.8581
Epoch 14/50
257/257 [==============================] - 25s 96ms/step - loss: 0.2814 - acc: 0.8762 - precision_22: 0.8818 - recall_22: 0.8669
Epoch 15/50
257/257 [==============================] - 24s 95ms/step - loss: 0.2654 - acc: 0.8846 - precision_22: 0.8850 - recall_22: 0.8824
Epoch 16/50
257/257 [==============================] - 25s 96ms/step - loss: 0.2586 - acc: 0.8857 - precision_22: 0.8908 - recall_22: 0.8775
Epoch 17/50
257/257 [==============================] - 25s 96ms/step - loss: 0.2374 - acc: 0.8975 - precision_22: 0.9013 - recall_22: 0.89121s - loss: 0.2386 - acc: 

257/257 [==============================] - 25s 98ms/step - loss: 0.2641 - acc: 0.8875 - precision_23: 0.8945 - recall_23: 0.8784
Epoch 17/50
257/257 [==============================] - 25s 99ms/step - loss: 0.2564 - acc: 0.8879 - precision_23: 0.8919 - recall_23: 0.8826
Epoch 18/50
257/257 [==============================] - 25s 97ms/step - loss: 0.2392 - acc: 0.8980 - precision_23: 0.9028 - recall_23: 0.8918
Epoch 19/50
257/257 [==============================] - 25s 98ms/step - loss: 0.2206 - acc: 0.9088 - precision_23: 0.9135 - recall_23: 0.9030
Epoch 20/50
257/257 [==============================] - 25s 98ms/step - loss: 0.2028 - acc: 0.9174 - precision_23: 0.9246 - recall_23: 0.9089
Epoch 21/50
257/257 [==============================] - 25s 98ms/step - loss: 0.1913 - acc: 0.9211 - precision_23: 0.9242 - recall_23: 0.9174
Epoch 22/50
257/257 [==============================] - 25s 98ms/step - loss: 0.1874 - acc: 0.9239 - precision_23: 0.9281 - recall_23: 0.9189
Epoch 23/50
257/257 [====

257/257 [==============================] - 26s 101ms/step - loss: 0.1781 - acc: 0.9254 - precision_24: 0.9260 - recall_24: 0.9240
Epoch 23/50
257/257 [==============================] - 26s 100ms/step - loss: 0.1683 - acc: 0.9329 - precision_24: 0.9345 - recall_24: 0.9306
Epoch 24/50
257/257 [==============================] - 26s 100ms/step - loss: 0.1639 - acc: 0.9333 - precision_24: 0.9358 - recall_24: 0.9298
Epoch 25/50
257/257 [==============================] - 26s 102ms/step - loss: 0.1462 - acc: 0.9413 - precision_24: 0.9439 - recall_24: 0.9379
Epoch 26/50
257/257 [==============================] - 26s 103ms/step - loss: 0.1507 - acc: 0.9377 - precision_24: 0.9383 - recall_24: 0.9364s - loss:
Epoch 27/50
257/257 [==============================] - 26s 101ms/step - loss: 0.1363 - acc: 0.9461 - precision_24: 0.9477 - recall_24: 0.9438
Epoch 28/50
257/257 [==============================] - 26s 101ms/step - loss: 0.1285 - acc: 0.9485 - precision_24: 0.9502 - recall_24: 0.9462
Epoch 29/

257/257 [==============================] - 26s 102ms/step - loss: 0.1317 - acc: 0.9465 - precision_25: 0.9468 - recall_25: 0.9462
Epoch 29/50
257/257 [==============================] - 26s 103ms/step - loss: 0.1162 - acc: 0.9539 - precision_25: 0.9562 - recall_25: 0.9513
Epoch 30/50
257/257 [==============================] - 26s 102ms/step - loss: 0.1117 - acc: 0.9559 - precision_25: 0.9559 - recall_25: 0.9559
Epoch 31/50
257/257 [==============================] - 26s 103ms/step - loss: 0.1076 - acc: 0.9582 - precision_25: 0.9581 - recall_25: 0.9583
Epoch 32/50
257/257 [==============================] - 27s 104ms/step - loss: 0.0965 - acc: 0.9658 - precision_25: 0.9677 - recall_25: 0.9637
Epoch 33/50
257/257 [==============================] - 26s 102ms/step - loss: 0.0911 - acc: 0.9651 - precision_25: 0.9656 - recall_25: 0.9644
Epoch 34/50
257/257 [==============================] - 26s 103ms/step - loss: 0.1037 - acc: 0.9615 - precision_25: 0.9624 - recall_25: 0.9605
Epoch 35/50
257/25

257/257 [==============================] - 26s 101ms/step - loss: 0.0921 - acc: 0.9660 - precision_26: 0.9657 - recall_26: 0.9669
Epoch 35/50
257/257 [==============================] - 26s 101ms/step - loss: 0.0904 - acc: 0.9652 - precision_26: 0.9688 - recall_26: 0.9618
Epoch 36/50
257/257 [==============================] - 26s 100ms/step - loss: 0.0850 - acc: 0.9673 - precision_26: 0.9671 - recall_26: 0.9678
Epoch 37/50
257/257 [==============================] - 26s 101ms/step - loss: 0.0805 - acc: 0.9690 - precision_26: 0.9716 - recall_26: 0.9666
Epoch 38/50
257/257 [==============================] - 26s 102ms/step - loss: 0.0969 - acc: 0.9645 - precision_26: 0.9665 - recall_26: 0.9628
Epoch 39/50
257/257 [==============================] - 27s 105ms/step - loss: 0.0825 - acc: 0.9708 - precision_26: 0.9724 - recall_26: 0.9695
Epoch 40/50
257/257 [==============================] - 26s 103ms/step - loss: 0.0784 - acc: 0.9710 - precision_26: 0.9705 - recall_26: 0.9720
Epoch 41/50
257/25

257/257 [==============================] - 26s 102ms/step - loss: 0.0733 - acc: 0.9744 - precision_27: 0.9735 - recall_27: 0.9754
Epoch 41/50
257/257 [==============================] - 26s 102ms/step - loss: 0.0703 - acc: 0.9739 - precision_27: 0.9730 - recall_27: 0.9749
Epoch 42/50
257/257 [==============================] - 26s 102ms/step - loss: 0.0668 - acc: 0.9746 - precision_27: 0.9749 - recall_27: 0.9742
Epoch 43/50
257/257 [==============================] - 26s 101ms/step - loss: 0.0701 - acc: 0.9721 - precision_27: 0.9722 - recall_27: 0.9720
Epoch 44/50
257/257 [==============================] - 26s 102ms/step - loss: 0.0608 - acc: 0.9780 - precision_27: 0.9785 - recall_27: 0.9773
Epoch 45/50
257/257 [==============================] - 27s 104ms/step - loss: 0.0627 - acc: 0.9781 - precision_27: 0.9776 - recall_27: 0.9786
Epoch 46/50
257/257 [==============================] - 26s 102ms/step - loss: 0.0543 - acc: 0.9806 - precision_27: 0.9815 - recall_27: 0.9798
Epoch 47/50
257/25

257/257 [==============================] - 26s 100ms/step - loss: 0.0539 - acc: 0.9809 - precision_28: 0.9807 - recall_28: 0.9810
Epoch 47/50
257/257 [==============================] - 26s 100ms/step - loss: 0.0503 - acc: 0.9842 - precision_28: 0.9844 - recall_28: 0.9839
Epoch 48/50
257/257 [==============================] - 26s 100ms/step - loss: 0.0558 - acc: 0.9792 - precision_28: 0.9793 - recall_28: 0.9790
Epoch 49/50
257/257 [==============================] - 27s 104ms/step - loss: 0.0462 - acc: 0.9853 - precision_28: 0.9851 - recall_28: 0.9854
Epoch 50/50
29/29 [==============================] - 1s 26ms/step - loss: 0.7829 - acc: 0.7840 - precision_28: 0.7633 - recall_28: 0.8341
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/50
257/257 [==============================] - 26s 99ms/step - loss: 0.5565 - acc: 0.7180 - precision_29: 0.7224 - recall_29: 0.7101
Epoch 2/50
257/257 [==============================] - 26s 100ms/step

29/29 [==============================] - 1s 28ms/step - loss: 0.7944 - acc: 0.8026 - precision_29: 0.7851 - recall_29: 0.8237


In [16]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.793885
test precision: 0.775448
test recall: 0.827498
test f1_score: 0.800373


In [17]:
#cnn+lstm

In [18]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
    
    modelCNNLSTM = Sequential() #cnn

    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    modelCNNLSTM.add(MaxPooling1D(pool_size=4))
    modelCNNLSTM.add(Dropout(0.25))
    modelCNNLSTM.add(LSTM(128))                            
    modelCNNLSTM.add(Flatten())
    modelCNNLSTM.add(Dense(1, activation='sigmoid'))
    modelCNNLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNNLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 11s 41ms/step - loss: 0.5588 - acc: 0.7124 - precision_30: 0.7266 - recall_30: 0.6858
Epoch 2/50
257/257 [==============================] - 10s 38ms/step - loss: 0.4348 - acc: 0.8014 - precision_30: 0.8163 - recall_30: 0.7805
Epoch 3/50
257/257 [==============================] - 10s 38ms/step - loss: 0.3468 - acc: 0.8448 - precision_30: 0.8564 - recall_30: 0.8304
Epoch 4/50
257/257 [==============================] - 9s 36ms/step - loss: 0.2747 - acc: 0.8829 - precision_30: 0.8888 - recall_30: 0.8767
Epoch 5/50
257/257 [==============================] - 9s 37ms/step - loss: 0.2061 - acc: 0.9207 - precision_30: 0.9235 - recall_30: 0.9184
Epoch 6/50
257/257 [==============================] - 9s 37ms/step - loss: 0.1488 - acc: 0.9417 - precision_30: 0.9410 - recall_30: 0.9431
Epoch 7/50
257/257 [==============================] - 9s 36ms/step

257/257 [==============================] - 9s 36ms/step - loss: 0.0228 - acc: 0.9921 - precision_31: 0.9927 - recall_31: 0.9915
Epoch 32/50
257/257 [==============================] - 10s 38ms/step - loss: 0.0133 - acc: 0.9953 - precision_31: 0.9954 - recall_31: 0.9951
Epoch 33/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0205 - acc: 0.9929 - precision_31: 0.9925 - recall_31: 0.9934
Epoch 34/50
257/257 [==============================] - 10s 38ms/step - loss: 0.0238 - acc: 0.9922 - precision_31: 0.9915 - recall_31: 0.9930
Epoch 35/50
257/257 [==============================] - 10s 37ms/step - loss: 0.0161 - acc: 0.9945 - precision_31: 0.9947 - recall_31: 0.9944
Epoch 36/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0140 - acc: 0.9956 - precision_31: 0.9956 - recall_31: 0.9956
Epoch 37/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0126 - acc: 0.9949 - precision_31: 0.9947 - recall_31: 0.9951
Epoch 38/50
257/257 [========

257/257 [==============================] - 10s 37ms/step - loss: 0.0572 - acc: 0.9756 - precision_33: 0.9752 - recall_33: 0.9761
Epoch 14/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0485 - acc: 0.9812 - precision_33: 0.9817 - recall_33: 0.9808
Epoch 15/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0485 - acc: 0.9825 - precision_33: 0.9832 - recall_33: 0.9817
Epoch 16/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0410 - acc: 0.9843 - precision_33: 0.9832 - recall_33: 0.9854
Epoch 17/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0392 - acc: 0.9854 - precision_33: 0.9856 - recall_33: 0.9851
Epoch 18/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0466 - acc: 0.9826 - precision_33: 0.9829 - recall_33: 0.9822
Epoch 19/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0374 - acc: 0.9871 - precision_33: 0.9880 - recall_33: 0.9861
Epoch 20/50
257/257 [==========

257/257 [==============================] - 9s 35ms/step - loss: 0.0920 - acc: 0.9642 - precision_35: 0.9643 - recall_35: 0.9639
Epoch 9/50
257/257 [==============================] - 9s 34ms/step - loss: 0.0766 - acc: 0.9699 - precision_35: 0.9679 - recall_35: 0.9719
Epoch 10/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0654 - acc: 0.9775 - precision_35: 0.9766 - recall_35: 0.9783
Epoch 11/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0545 - acc: 0.9787 - precision_35: 0.9781 - recall_35: 0.9792
Epoch 12/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0581 - acc: 0.9761 - precision_35: 0.9747 - recall_35: 0.9775
Epoch 13/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0443 - acc: 0.9822 - precision_35: 0.9815 - recall_35: 0.9829
Epoch 14/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0482 - acc: 0.9811 - precision_35: 0.9798 - recall_35: 0.9824
Epoch 15/50
257/257 [============

257/257 [==============================] - 10s 37ms/step - loss: 0.0215 - acc: 0.9922 - precision_36: 0.9929 - recall_36: 0.9914
Epoch 37/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0150 - acc: 0.9939 - precision_36: 0.9948 - recall_36: 0.9929
Epoch 38/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0219 - acc: 0.9927 - precision_36: 0.9938 - recall_36: 0.9914
Epoch 39/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0286 - acc: 0.9892 - precision_36: 0.9909 - recall_36: 0.9872
Epoch 40/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0160 - acc: 0.9948 - precision_36: 0.9963 - recall_36: 0.9931
Epoch 41/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0310 - acc: 0.9892 - precision_36: 0.9894 - recall_36: 0.9887
Epoch 42/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0176 - acc: 0.9943 - precision_36: 0.9958 - recall_36: 0.9926
Epoch 00042: early stopping
29/

257/257 [==============================] - 9s 36ms/step - loss: 0.0328 - acc: 0.9865 - precision_38: 0.9870 - recall_38: 0.9858
Epoch 22/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0295 - acc: 0.9877 - precision_38: 0.9885 - recall_38: 0.9868
Epoch 23/50
257/257 [==============================] - 9s 37ms/step - loss: 0.0380 - acc: 0.9862 - precision_38: 0.9870 - recall_38: 0.9854
Epoch 24/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0296 - acc: 0.9882 - precision_38: 0.9876 - recall_38: 0.9888
Epoch 25/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0355 - acc: 0.9860 - precision_38: 0.9866 - recall_38: 0.9854
Epoch 26/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0313 - acc: 0.9901 - precision_38: 0.9914 - recall_38: 0.9888
Epoch 27/50
257/257 [==============================] - 9s 36ms/step - loss: 0.0266 - acc: 0.9897 - precision_38: 0.9902 - recall_38: 0.9890
Epoch 28/50
257/257 [===========

In [19]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.763752
test precision: 0.750416
test recall: 0.794080
test f1_score: 0.770320
